In [30]:
import numpy as np
from sklearn.metrics import brier_score_loss
import os
import random
import pandas as pd


In [31]:

if not os.getcwd().endswith('Football Forecasting Version 2'):
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

dir_path = os.getcwd()

# CREATES DICTIONARY OF THREE LETTER ON CLUBS

res = []
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a folder
    if os.path.isdir(os.path.join(dir_path, path)) and 'Matchweek' in path:
        res.append(path)

three_let_dict = {}
for matchweek_folder in res:
    os.chdir(fr'{dir_path}\{matchweek_folder}')
    a = pd.read_csv(random.choice(
        os.listdir(fr'{dir_path}\{matchweek_folder}')))
    for index, row in a.iterrows():
        three_let_dict[row['Club'].strip()] = row['Code']
    os.chdir(dir_path)


three_let_dict['Tottenham'] = three_let_dict.pop('Tottenham Hotspur')
three_let_dict['Brighton'] = three_let_dict.pop('Brighton and Hove Albion')
three_let_dict['Manchester Utd'] = three_let_dict.pop('Manchester United')
three_let_dict['Newcastle Utd'] = three_let_dict.pop('Newcastle United')
three_let_dict['West Ham'] = three_let_dict.pop('West Ham United')
three_let_dict['Wolves'] = three_let_dict.pop('Wolverhampton Wanderers')
three_let_dict["Nott'ham Forest"] = three_let_dict.pop('Nottingham Forest')
three_let_dict["Sheffield Utd"] = three_let_dict.pop('Sheffield United')
three_let_dict["Huddersfield"] = three_let_dict.pop('Huddersfield Town')
three_let_dict["West Brom"] = three_let_dict.pop('West Bromwich Albion')

In [32]:
res = []
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a folder
    if os.path.isdir(os.path.join(dir_path, path)) and 'Fixture' in path:
        res.append(path)
        
df = pd.DataFrame()
for fixtures in res:
    os.chdir(fr'{dir_path}\{fixtures}')
    folder = os.getcwd()
    list_of_files = os.listdir()
    for i in list_of_files:
        b = pd.read_csv(fr'{folder}\{i}', index_col=0)
        b['Date'] = pd.to_datetime(b['Date'])
        b['Club'] = b['Club'].map(three_let_dict)
        b['Opponent'] = b['Opponent'].map(three_let_dict)
        b = b[b['Date'] < np.datetime64('today') ].iloc[[-1]]
        df = pd.concat([df, b])
    os.chdir(dir_path)
df = df.reset_index(drop=True)
df = df.sort_values(by=['Date'])
df = df.reset_index(drop=True)
display(df)

,Date,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Club
0,2018-05-13,Away,W,1.0,0.0,HUD,2.0,1.7,55.0,ARS
1,2018-05-13,Home,W,3.0,1.0,EVE,1.1,1.6,57.0,WHU
2,2018-05-13,Away,L,0.0,2.0,CRY,0.2,2.2,41.0,WBA
3,2018-05-13,Away,L,0.0,1.0,MUN,0.6,0.4,48.0,WAT
4,2018-05-13,Home,W,5.0,4.0,LEI,1.8,1.4,64.0,TOT
...,...,...,...,...,...,...,...,...,...,...
115,2023-03-19,Home,NaN,NaN,NaN,MUN,NaN,NaN,NaN,BHA
116,2023-03-19,Home,NaN,NaN,NaN,WHU,NaN,NaN,NaN,MCI
117,2023-03-19,Away,NaN,NaN,NaN,BHA,NaN,NaN,NaN,MUN
118,2023-03-19,Home,W,4.0,1.0,CRY,1.6,0.8,62.0,ARS


In [33]:
os.chdir(fr'{dir_path}\2022_2023_Match_Predictions')

predictions_df = pd.DataFrame()
for i in os.listdir():
    if 'predictions' in i:
        a = pd.read_csv(i, index_col = 0)
        predictions_df = pd.concat([predictions_df, a])
predictions_df = predictions_df.reset_index(drop=True)
predictions_df = predictions_df.dropna()
predictions_df['Date'] = pd.to_datetime(predictions_df['Date'])
predictions_df = predictions_df.drop_duplicates(subset=['Club_x','Opp_x','Date'],keep = 'last')

In [34]:
measure_df = pd.merge(df,predictions_df, left_on = ['Date','Club','Opponent'], right_on = ['Date','Club_x','Opp_x'] )
measure_df['Game'] = measure_df['Club'] + measure_df['Opponent']
nice = measure_df['Game'].to_list()
new_list = []
for i in nice:
    if i[0] > i[3]:
        new_list.append(i[3:] + i[0:3])
    else:
        new_list.append(i)
new_list = list(set(new_list))
measure_df = measure_df[measure_df['Game'].isin(new_list)]
measure_df = measure_df.dropna()

In [35]:
w_list = []
d_list = []
l_list = []
for i in measure_df['Result']:
    if i == 'W':
        w_list.append(1)
        d_list.append(0)
        l_list.append(0)
    elif i == 'D':
        w_list.append(0)
        d_list.append(1)
        l_list.append(0)
    else:
        w_list.append(0)
        d_list.append(0)
        l_list.append(1)
measure_df['win'] = w_list
measure_df['draw'] = d_list
measure_df['loss'] = l_list
measure_df = measure_df[['Date','Club','Opponent','Result','win_prob','draw_prob','loss_prob']]
display(measure_df)

,Date,Club,Opponent,Result,win_prob,draw_prob,loss_prob
2,2023-03-18,SOU,TOT,D,0.147211,0.214246,0.638543
3,2023-03-18,LEE,WOL,W,0.368169,0.275028,0.356803
6,2023-03-18,CHE,EVE,D,0.644182,0.213885,0.141933
7,2023-03-18,BRE,LEI,D,0.519392,0.240701,0.239907
9,2023-03-18,AVL,BOU,W,0.580742,0.231900,0.187358
16,2023-03-19,ARS,CRY,W,0.660651,0.203702,0.135647


In [36]:

measure_df = measure_df.style.background_gradient(axis=1, vmin=0, vmax=0.9)

display(measure_df)

,Date,Club,Opponent,Result,win_prob,draw_prob,loss_prob
2,2023-03-18 00:00:00,SOU,TOT,D,0.147211,0.214246,0.638543
3,2023-03-18 00:00:00,LEE,WOL,W,0.368169,0.275028,0.356803
6,2023-03-18 00:00:00,CHE,EVE,D,0.644182,0.213885,0.141933
7,2023-03-18 00:00:00,BRE,LEI,D,0.519392,0.240701,0.239907
9,2023-03-18 00:00:00,AVL,BOU,W,0.580742,0.231900,0.187358
16,2023-03-19 00:00:00,ARS,CRY,W,0.660651,0.203702,0.135647


In [37]:
y_true = w_list + d_list + l_list
y_prob = list(measure_df['win_prob']) + list(measure_df['draw_prob']) + list(measure_df['loss_prob'])

brier_score_loss(y_true,y_prob) * 3

TypeError: 'Styler' object is not subscriptable